In [1]:
] activate

 Activating environment at `~/Dropbox/EMAC/DCSideBatteryModeling/Project.toml`


In [17]:
using LinearAlgebra
using OrdinaryDiffEq #Gets the solvers
using Plots
plotly()
using PowerSystems
using DelimitedFiles

include(joinpath(pwd(), "DCSideBatteryModeling", "DCSideBatteryModeling.jl"))

In [5]:
include(joinpath(pwd(), "data","make_data.jl"))

┌ Info: The PSS(R)E parser currently supports buses, loads, shunts, generators, branches, transformers, and dc lines
└ @ PowerSystems /home/ciaran/.julia/packages/PowerSystems/kfHHW/src/parsers/pm_io/common.jl:26
┌ Info: Constructing System from Power Models
│   data["name"] = omib
│   data["source_type"] = pti
└ @ PowerSystems /home/ciaran/.julia/packages/PowerSystems/kfHHW/src/parsers/power_models_data.jl:39
┌ Info: Created InMemoryTimeSeriesStorage
└ @ InfrastructureSystems /home/ciaran/.julia/packages/InfrastructureSystems/WC1sp/src/in_memory_time_series_storage.jl:24
┌ Info: Reading bus data
└ @ PowerSystems /home/ciaran/.julia/packages/PowerSystems/kfHHW/src/parsers/power_models_data.jl:140
┌ Info: Reading generator data
└ @ PowerSystems /home/ciaran/.julia/packages/PowerSystems/kfHHW/src/parsers/power_models_data.jl:390
┌ Info: Reading branch data
└ @ PowerSystems /home/ciaran/.julia/packages/PowerSystems/kfHHW/src/parsers/power_models_data.jl:543
┌ Info: Reading branch data
└ @

LoadError: UndefKeywordError: keyword argument basepower not assigned

In [6]:
omib_sys = System(joinpath(pwd(), "data", "OMIB_inverterDCside.json"))

ErrorException: type NamedTuple has no field validation_descriptor_file

In [7]:
fb = 60 # Base frequency. 
_ωb = 2 * pi * fb # Base angular velocity 
Sb = 0.2e6 # Base Power
# System base for AC side
Vb = 480 # Base AC voltage
Ib = Sb / Vb # Base AC Current 
Zb = Vb / Ib # Base Impedancec
Lb = Zb / _ωb # Base Inductance
Cb = 1 / (Zb * _ωb) # Base AC capcitacne
# System base for DC side
Vb_dc = (sqrt(8)/sqrt(3))*Vb # Base DC voltage
Ib_dc = Sb / Vb_dc # Base DC Current
Zb_dc = Vb_dc / Ib_dc # Base DC Impedance
Lb_dc = Zb_dc / _ωb # Base DC Inductance
Cb_dc = 1 / (Zb_dc * _ωb) # Base DC Capcitance

0.0008634708284065503

## Instantiate ODE Models

In [8]:
parameter_mapping = instantiate_parameters(omib_sys)
M_4th= instantiate_4th_order_model(omib_sys)
M_2nd= instantiate_2nd_order_model(omib_sys)
M_0th= instantiate_0th_order_model(omib_sys)

ModelOperatingPoint


In [9]:
_parameter_values = [x.second for x in parameter_mapping]; # Extract list of parameter values

In [10]:
cap_size = 2000
_parameter_values[38]=cap_size*1e-6/Cb_dc # DC-link capacitance
_parameter_values[44]=0 # K_pred=0 for initial simulations

0

## Re-calculate initial conditons

In [11]:
M_4th(_parameter_values)
M_2nd(_parameter_values)
M_0th(_parameter_values)

21-element Array{Float64,1}:
  1.0067935237422023
 -0.040061818677185806
  0.2032828250035323
  0.05850652191227816
  0.2003183879749555
 -0.01599274198690052
  0.2023199540292933
  0.008076270122532621
  0.00027217172279205904
 -2.172926900394095e-5
  0.07044778826693796
 -0.0027892516861153125
  1.084409521544636
  0.42943516874521637
 -7.084994987786322e-19
  0.054666706517905735
 -1.453234466653399e-31
  4.600046756145388e-9
  1.97215226305253e-31
  0.0
  1.437514611295434e-13

## Make copy of parameters and change value of load at t=0

In [12]:
disturbance=deepcopy(_parameter_values)
disturbance[9]=0.7 # Load step change from 0.2 p.u. to 0.7 p.u.

0.7

## Instantiate and simualte 4$^{th}$-, 2$^{nd}$- and 0$^{th}$-order model for non-optimized gains

In [18]:
sim_4th_nonopt = instantiate_4th_order_dae(disturbance, M_4th; tspan = (0.0, 0.05))
sol_4th_nonopt = solve(sim_4th_nonopt, Rodas5(), saveat = 0.0001);

In [19]:
sim_2nd_nonopt = instantiate_2nd_order_dae(disturbance, M_2nd; tspan = (0.0, 0.05))
sol_2nd_nonopt = solve(sim_2nd_nonopt, Rodas5(), saveat = 0.0001);

In [20]:
sim_0th_nonopt = instantiate_0th_order_dae(disturbance, M_0th; tspan = (0.0, 0.05))
sol_0th_nonopt = solve(sim_0th_nonopt, Rodas5(), saveat = 0.0001);

## Plotting DC-voltage for all three models

In [21]:
plot(sol_4th_nonopt, vars=(0,13), label="4th-order", ylabel="DC-link Voltage (p.u.)")
plot!(sol_2nd_nonopt, vars=(0,13), label="2nd-order")
plot!(sol_0th_nonopt, vars=(0,13), label="0th-order")

<!DOCTYPE html>
 
 
 Plots.jl

## Save results to array and write to .csv file

In [22]:
untuned_4th = [sol_4th_nonopt.t transpose(Array(sol_4th_nonopt))]
untuned_2nd = [sol_2nd_nonopt.t transpose(Array(sol_2nd_nonopt))]
untuned_0th = [sol_0th_nonopt.t transpose(Array(sol_0th_nonopt))];

In [23]:
fileName=string("results/dae_sim/untuned_4th_", cap_size, "uF.csv")
writedlm( fileName,  untuned_4th, ',')
fileName=string("results/dae_sim/untuned_2nd_", cap_size, "uF.csv")
writedlm( fileName,  untuned_2nd, ',')
fileName=string("results/dae_sim/untuned_0th_", cap_size, "uF.csv")
writedlm( fileName,  untuned_0th, ',')

## Load optimal gains from parameter_search.ipynb

In [24]:
fileName=string("results/stable_gains/capSize_", cap_size, "uF.csv") 
gains = readdlm(fileName, ',') # Loading set of stable gains 
fileName=string("results/optimal_gains/capSize_", cap_size, "uF.csv")
optIndex = readdlm( fileName, ',') # Loading index of optimal set of gains

2×1 Array{Float64,2}:
 4443.0
    0.06802762588059204

## Set controller gains to optimal values

In [25]:
_parameter_values[40]=gains[Int(optIndex[1]),1] #K_p^v
_parameter_values[41]=gains[Int(optIndex[1]),2] #K_i^v
_parameter_values[42]=gains[Int(optIndex[1]),3] #K_p^i
_parameter_values[43]=gains[Int(optIndex[1]),4] #K_i^i
_parameter_values[44]=0 #K_pred

0

## Re-calculate initial conditons

In [26]:
M_4th(_parameter_values)
M_2nd(_parameter_values)
M_0th(_parameter_values)

21-element Array{Float64,1}:
  1.0067935237422023
 -0.040061818677185806
  0.2032828250035323
  0.058506521912278175
  0.2003183879749555
 -0.01599274198690052
  0.2023199540292933
  0.008076270122532618
  0.0002721717227920591
 -2.1729269003940933e-5
  0.07044778826693796
 -0.002789251686115313
  1.084409521544636
  0.4294351687452164
 -6.811271362385676e-19
  0.05514670686781933
  2.5036080038079005e-38
  4.600046756145387e-9
 -4.3790577010150533e-47
  0.0
  1.4375146112954338e-13

## Make copy of parameters and change value of load at t=0

In [27]:
disturbance=deepcopy(_parameter_values)
disturbance[9]=0.7 # Load step change from 0.2 p.u. to 0.7 p.u.

0.7

## Instantiate and simulate 4$^{th}$-, 2$^{nd}$- and 0$^{th}$-order model with optimized gains

In [28]:
sim_4th_opt = instantiate_4th_order_dae(disturbance, M_4th; tspan = (0.0, 0.02))
sol_4th_opt = solve(sim_4th_opt, Rodas5(), saveat = 0.0001);

In [29]:
sim_2nd_opt = instantiate_2nd_order_dae(disturbance, M_2nd; tspan = (0.0, 0.02))
sol_2nd_opt = solve(sim_2nd_opt, Rodas5(), saveat = 0.0001);

In [30]:
sim_0th_opt = instantiate_0th_order_dae(disturbance, M_0th; tspan = (0.0, 0.02))
sol_0th_opt = solve(sim_0th_opt, Rodas5(), saveat = 0.0001);

## Plotting DC-voltage for all three models

In [31]:
plot(sol_4th_opt, vars=(0,13), label="4th-order", ylabel="DC-link Voltage (p.u.)")
plot!(sol_2nd_opt, vars=(0,13), label="2nd-order")
plot!(sol_0th_opt, vars=(0,13), label="0th-order")

<!DOCTYPE html>
 
 
 Plots.jl

## Save results to array and write to .csv file

In [32]:
tuned_4th = [sol_4th_opt.t transpose(Array(sol_4th_opt))]
tuned_2nd = [sol_2nd_opt.t transpose(Array(sol_2nd_opt))]
tuned_0th = [sol_0th_opt.t transpose(Array(sol_0th_opt))];

In [33]:
fileName=string("results/dae_sim/tuned_4th_", cap_size, "uF.csv")
writedlm( fileName,  tuned_4th, ',')
fileName=string("results/dae_sim/tuned_2nd_", cap_size, "uF.csv")
writedlm( fileName,  tuned_2nd, ',')
fileName=string("results/dae_sim/tuned_0th_", cap_size, "uF.csv")
writedlm( fileName,  tuned_0th, ',')

# Performance imporvement from increasing predictive gain

In [34]:
_parameter_values[44]=0 #K_pred
M_0th(_parameter_values) # Recalcualte initial conditons 

disturbance=deepcopy(_parameter_values)
disturbance[9]=0.7 # Load step change from 0.2 p.u. to 0.7 p.u.

sim_k0 = instantiate_0th_order_dae(disturbance, M_0th; tspan = (0.0, 0.03))
sol_k0 = solve(sim_k0, Rodas5(), saveat = 0.0001);

In [35]:
_parameter_values[44]=1 #K_pred
M_0th(_parameter_values) # Recalcualte initial conditons 

disturbance=deepcopy(_parameter_values)
disturbance[9]=0.7 # Load step change from 0.2 p.u. to 0.7 p.u.

sim_k1 = instantiate_0th_order_dae(disturbance, M_0th; tspan = (0.0, 0.02))
sol_k1 = solve(sim_k1, Rodas5(), saveat = 0.0001);

In [36]:
_parameter_values[44]=2 #K_pred
M_0th(_parameter_values) # Recalcualte initial conditons 

disturbance=deepcopy(_parameter_values)
disturbance[9]=0.7 # Load step change from 0.2 p.u. to 0.7 p.u.

sim_k2 = instantiate_0th_order_dae(disturbance, M_0th; tspan = (0.0, 0.03))
sol_k2 = solve(sim_k2, Rodas5(), saveat = 0.0001);

In [37]:
_parameter_values[44]=5 #K_pred
M_0th(_parameter_values) # Recalcualte initial conditons 

disturbance=deepcopy(_parameter_values)
disturbance[9]=0.7 # Load step change from 0.2 p.u. to 0.7 p.u.

sim_k5 = instantiate_0th_order_dae(disturbance, M_0th; tspan = (0.0, 0.03))
sol_k5 = solve(sim_k5, Rodas5(), saveat = 0.0001);

## Plotting DC-voltage for various predictive gain

In [38]:
plot(sol_k0, vars=(0,13), xlims = (0,0.025), label="k_pred=0", ylabel="DC-link Voltage (p.u.)")
plot!(sol_k1, vars=(0,13), xlims = (0,0.025), label="k_pred=1")
plot!(sol_k2, vars=(0,13), xlims = (0,0.025), label="k_pred=2")
plot!(sol_k5, vars=(0,13), xlims = (0,0.025), label="k_pred=5")

<!DOCTYPE html>
 
 
 Plots.jl

## Save results to array and write to .csv file

In [39]:
kpred_0 = [sol_k0.t transpose(Array(sol_k0))]
kpred_1 = [sol_k1.t transpose(Array(sol_k1))]
kpred_2 = [sol_k2.t transpose(Array(sol_k2))];
kpred_5 = [sol_k5.t transpose(Array(sol_k5))];

In [40]:
writedlm( "results/dae_sim/kpred_0.csv",  kpred_0, ',')
writedlm( "results/dae_sim/kpred_1.csv",  kpred_1, ',')
writedlm( "results/dae_sim/kpred_2.csv",  kpred_2, ',')
writedlm( "results/dae_sim/kpred_5.csv",  kpred_5, ',')

## Calculating reduction in voltage nadir

In [41]:
nadir_reduction = 100*(minimum(kpred_5[:,14])-minimum(kpred_0[:,14]))/(1.1-minimum(kpred_0[:,14]))

10.923412749594556

## Sweeping DC_link Capacitor size and predictive gain to generate heatmap

In [35]:
heat_map_values=reshape([],0,3);
cap_range = 2000:500:7000 # DC-link capacitor sizing
kpred_range = 0:0.25:5 # Range of predictive gain
for cap_size in cap_range
    for k in kpred_range
        _parameter_values[38]=cap_size*1e-6/Cb_dc # Adjusting DC-link capacitor size
        fileName=string("results/stable_gains/capSize_", cap_size, "uF.csv")
        gains = readdlm(fileName, ',', Float64) # Loading set of stable gains
        fileName=string("results/optimal_gains/capSize_", cap_size, "uF.csv")
        optIndex = readdlm(fileName, ',', Float64) # Loading index of optimal gain
        _parameter_values[40]=gains[Int(optIndex[1]),1] #K_p^v
        _parameter_values[41]=gains[Int(optIndex[1]),2] #K_i^v
        _parameter_values[42]=gains[Int(optIndex[1]),3] #K_p^i
        _parameter_values[43]=gains[Int(optIndex[1]),4] #K_i^i
        _parameter_values[44]=k #K_pred
        M_0th(_parameter_values) #Initializing 0th order model

        disturbance=deepcopy(_parameter_values)
        disturbance[9]=0.7 # Step change in load from 0.2 p.u. to 0.7 p.u.

        sim = instantiate_0th_order_dae(disturbance, M_0th; tspan = (0.0, 0.02))
        sol = solve(sim, Rodas5(), saveat = 0.0001);
        solArray = Array(sol)
        nadir=100*maximum(_parameter_values[39].-solArray[13,:]) # Calculating nadir drop in % of Base Votlage 
        
        heat_map_values = [heat_map_values; [cap_size/1e3 k round(nadir, digits=3)]] # Concatenating results
    end
end
writedlm( "results/dae_sim/heat_map_matrix.csv",  heat_map_values, ',') # Writing array to .csv file

UndefVarError: UndefVarError: _parameter_values not defined